# Data analysis - BoligPortalen

In [19]:
# Import packages:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

In [20]:
# import data
bp_df = pd.read_csv('boligportalen_df.csv')
bp_df

,Adress,Monthly rent,Aconto,Square meters,Rooms,Floor,Property type,Furnished,Shareable,Pets allowed,Elevator,Senior frendly,Only for students,Balcony/Terrace,Parking,Energy label,Description,Link
0,"Sct Knuds Alle, 6740 Bramming - 1. sal",7.712 kr.,868 kr.,109 m²,4.0,1.,Lejlighed,Nej,Ja,Nej,Nej,Nej,Nej,Nej,Nej,NaN,Flyt i lækkert nybyggeri centralt i Bramming\n...,https://www.boligportal.dk/lejligheder/brammin...
1,"Strøget, 7430 Ikast - 2. sal",5.725 kr.,900 kr.,91 m²,3.0,2.,Lejlighed,Nej,Ja,Nej,Nej,Nej,Nej,Ja,Nej,E,Central placeret 3 værelses lejlighed udlejes ...,https://www.boligportal.dk/lejligheder/ikast/9...
2,"Lykkehåbs Alle, 8722 Hedensted - 1. sal",8.545 kr.,1.000 kr.,83 m²,3.0,1.,Lejlighed,Nej,Nej,Ja,Ja,Ja,Nej,Ja,Ja,A,Lejlighederne er bygget i elegant skandinavisk...,https://www.boligportal.dk/lejligheder/hedenst...
3,"Annagade, 4220 Korsør - 1. sal",7.000 kr.,1.100 kr.,98 m²,3.0,1.,Lejlighed,Nej,Nej,Nej,Nej,Nej,Nej,Nej,Nej,NaN,Stor 3 værelses lejlighed beliggende på Annaga...,https://www.boligportal.dk/lejligheder/kors%C3...
4,"Ramsvej, 8800 Viborg - 2. sal",6.990 kr.,1.000 kr.,100 m²,3.0,2.,Lejlighed,Nej,Ja,Nej,Nej,Nej,Nej,Nej,Nej,NaN,Rigtig fin lejlighed med central beliggenhed ...,https://www.boligportal.dk/lejligheder/viborg/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14233,"Reberbansgade, 9000 Aalborg, Aalborg - 3. sal",4.875 kr.,500 kr.,56 m²,2.0,3.,Lejlighed,Nej,Nej,Nej,Nej,Nej,Nej,Nej,Nej,NaN,Dejlig 2 værelses lejlighed i centralt beligge...,https://www.boligportal.dk/lejligheder/aalborg...
14234,"Parkvej, 4700 Næstved - 5. sal",6.500 kr.,700 kr.,48 m²,2.0,5.,Lejlighed,Ja,Nej,Ja,Ja,Ja,Ja,Ja,Ja,C,Lejligheden har en super beliggenhed og udleje...,https://www.boligportal.dk/lejligheder/n%C3%A6...
14235,"Skolegade, 9300 Sæby",4.200 kr.,700 kr.,83 m²,2.0,-,Lejlighed,Nej,Nej,Nej,Nej,Nej,Nej,Ja,Ja,NaN,Stor rummelig 2-værelses lejlighed med eget kø...,https://www.boligportal.dk/lejligheder/s%C3%A6...
14236,"Danmarksgade, 9000 Aalborg, Aalborg - 2. sal",5.675 kr.,600 kr.,97 m²,3.0,2.,Lejlighed,Nej,Nej,Nej,Nej,Nej,Nej,Nej,Nej,A,Særdeles velbeliggende 3-vær. midt i Aalborg.\...,https://www.boligportal.dk/lejligheder/aalborg...


In [21]:
# transform data
bp_df['Monthly rent'] = bp_df['Monthly rent'].str.replace(' kr.', '').str.replace('.', '')
bp_df['Aconto'] = bp_df['Aconto'].str.replace(' kr.', '').str.replace('.', '')
bp_df['Square meters'] = bp_df['Square meters'].str.replace(' m²', '').str.replace('.', '')
bp_df['Rooms'] = bp_df['Rooms'].astype(str).str.replace('.0', '')
bp_df['Floor'] = bp_df['Floor'].astype(str).str.replace('.', '')
bp_df['Floor'] = bp_df['Floor'].str.replace('Stuen', '0')
bp_df['Floor'] = bp_df['Floor'].str.replace('Kælder', '-1')


bp_df

,Adress,Monthly rent,Aconto,Square meters,Rooms,Floor,Property type,Furnished,Shareable,Pets allowed,Elevator,Senior frendly,Only for students,Balcony/Terrace,Parking,Energy label,Description,Link
0,"Sct Knuds Alle, 6740 Bramming - 1. sal",7712,868,109,4,1,Lejlighed,Nej,Ja,Nej,Nej,Nej,Nej,Nej,Nej,NaN,Flyt i lækkert nybyggeri centralt i Bramming\n...,https://www.boligportal.dk/lejligheder/brammin...
1,"Strøget, 7430 Ikast - 2. sal",5725,900,91,3,2,Lejlighed,Nej,Ja,Nej,Nej,Nej,Nej,Ja,Nej,E,Central placeret 3 værelses lejlighed udlejes ...,https://www.boligportal.dk/lejligheder/ikast/9...
2,"Lykkehåbs Alle, 8722 Hedensted - 1. sal",8545,1000,83,3,1,Lejlighed,Nej,Nej,Ja,Ja,Ja,Nej,Ja,Ja,A,Lejlighederne er bygget i elegant skandinavisk...,https://www.boligportal.dk/lejligheder/hedenst...
3,"Annagade, 4220 Korsør - 1. sal",7000,1100,98,3,1,Lejlighed,Nej,Nej,Nej,Nej,Nej,Nej,Nej,Nej,NaN,Stor 3 værelses lejlighed beliggende på Annaga...,https://www.boligportal.dk/lejligheder/kors%C3...
4,"Ramsvej, 8800 Viborg - 2. sal",6990,1000,100,3,2,Lejlighed,Nej,Ja,Nej,Nej,Nej,Nej,Nej,Nej,NaN,Rigtig fin lejlighed med central beliggenhed ...,https://www.boligportal.dk/lejligheder/viborg/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14233,"Reberbansgade, 9000 Aalborg, Aalborg - 3. sal",4875,500,56,2,3,Lejlighed,Nej,Nej,Nej,Nej,Nej,Nej,Nej,Nej,NaN,Dejlig 2 værelses lejlighed i centralt beligge...,https://www.boligportal.dk/lejligheder/aalborg...
14234,"Parkvej, 4700 Næstved - 5. sal",6500,700,48,2,5,Lejlighed,Ja,Nej,Ja,Ja,Ja,Ja,Ja,Ja,C,Lejligheden har en super beliggenhed og udleje...,https://www.boligportal.dk/lejligheder/n%C3%A6...
14235,"Skolegade, 9300 Sæby",4200,700,83,2,-,Lejlighed,Nej,Nej,Nej,Nej,Nej,Nej,Ja,Ja,NaN,Stor rummelig 2-værelses lejlighed med eget kø...,https://www.boligportal.dk/lejligheder/s%C3%A6...
14236,"Danmarksgade, 9000 Aalborg, Aalborg - 2. sal",5675,600,97,3,2,Lejlighed,Nej,Nej,Nej,Nej,Nej,Nej,Nej,Nej,A,Særdeles velbeliggende 3-vær. midt i Aalborg.\...,https://www.boligportal.dk/lejligheder/aalborg...


In [23]:
# Mapping for 'Nej' to 0 and 'Ja' to 1
mapping = {'Nej': 0, 'Ja': 1}

# List of columns to be converted
columns_to_convert = ['Furnished', 'Shareable', 'Pets allowed', 'Elevator', 'Senior frendly',
                      'Only for students', 'Balcony/Terrace', 'Parking']

# Apply mapping to specified columns
bp_df[columns_to_convert] = bp_df[columns_to_convert].replace(mapping)

# Convert the columns to integer data type
##bp_df[columns_to_convert] = bp_df[columns_to_convert].astype(int)

# Replace "-" with NaN in the entire DataFrame
bp_df.replace("-", np.nan, inplace=True)

# Display the updated DataFrame
bp_df


,Adress,Monthly rent,Aconto,Square meters,Rooms,Floor,Property type,Furnished,Shareable,Pets allowed,Elevator,Senior frendly,Only for students,Balcony/Terrace,Parking,Energy label,Description,Link
0,"Sct Knuds Alle, 6740 Bramming - 1. sal",7712,868,109,4,1,Lejlighed,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Flyt i lækkert nybyggeri centralt i Bramming\n...,https://www.boligportal.dk/lejligheder/brammin...
1,"Strøget, 7430 Ikast - 2. sal",5725,900,91,3,2,Lejlighed,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,E,Central placeret 3 værelses lejlighed udlejes ...,https://www.boligportal.dk/lejligheder/ikast/9...
2,"Lykkehåbs Alle, 8722 Hedensted - 1. sal",8545,1000,83,3,1,Lejlighed,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,A,Lejlighederne er bygget i elegant skandinavisk...,https://www.boligportal.dk/lejligheder/hedenst...
3,"Annagade, 4220 Korsør - 1. sal",7000,1100,98,3,1,Lejlighed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Stor 3 værelses lejlighed beliggende på Annaga...,https://www.boligportal.dk/lejligheder/kors%C3...
4,"Ramsvej, 8800 Viborg - 2. sal",6990,1000,100,3,2,Lejlighed,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Rigtig fin lejlighed med central beliggenhed ...,https://www.boligportal.dk/lejligheder/viborg/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14233,"Reberbansgade, 9000 Aalborg, Aalborg - 3. sal",4875,500,56,2,3,Lejlighed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Dejlig 2 værelses lejlighed i centralt beligge...,https://www.boligportal.dk/lejligheder/aalborg...
14234,"Parkvej, 4700 Næstved - 5. sal",6500,700,48,2,5,Lejlighed,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,C,Lejligheden har en super beliggenhed og udleje...,https://www.boligportal.dk/lejligheder/n%C3%A6...
14235,"Skolegade, 9300 Sæby",4200,700,83,2,NaN,Lejlighed,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,Stor rummelig 2-værelses lejlighed med eget kø...,https://www.boligportal.dk/lejligheder/s%C3%A6...
14236,"Danmarksgade, 9000 Aalborg, Aalborg - 2. sal",5675,600,97,3,2,Lejlighed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,Særdeles velbeliggende 3-vær. midt i Aalborg.\...,https://www.boligportal.dk/lejligheder/aalborg...


In [31]:
# Assuming bp_df is the DataFrame containing your data
# Replace 'bp_df' with the actual name of your DataFrame

# Columns to convert to float
float_columns = [
    "Monthly rent", "Aconto", "Square meters", "Rooms", "Floor",
    "Furnished", "Shareable", "Pets allowed", "Elevator", "Senior frendly",
    "Only for students", "Balcony/Terrace", "Parking", "Energy label"
]

# Clean and convert columns to float
for col in float_columns:
    if bp_df[col].dtype == object:
        bp_df[col] = pd.to_numeric(bp_df[col].str.replace(',', ''), errors='coerce')

# Now the specified columns are cleaned and converted to float data type


In [32]:
bp_df

,Adress,Monthly rent,Aconto,Square meters,Rooms,Floor,Property type,Furnished,Shareable,Pets allowed,Elevator,Senior frendly,Only for students,Balcony/Terrace,Parking,Energy label,Description,Link
0,"Sct Knuds Alle, 6740 Bramming - 1. sal",7712.0,868.0,109.0,4.0,1.0,Lejlighed,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Flyt i lækkert nybyggeri centralt i Bramming\n...,https://www.boligportal.dk/lejligheder/brammin...
1,"Strøget, 7430 Ikast - 2. sal",5725.0,900.0,91.0,3.0,2.0,Lejlighed,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,Central placeret 3 værelses lejlighed udlejes ...,https://www.boligportal.dk/lejligheder/ikast/9...
2,"Lykkehåbs Alle, 8722 Hedensted - 1. sal",8545.0,1000.0,83.0,3.0,1.0,Lejlighed,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,Lejlighederne er bygget i elegant skandinavisk...,https://www.boligportal.dk/lejligheder/hedenst...
3,"Annagade, 4220 Korsør - 1. sal",7000.0,1100.0,98.0,3.0,1.0,Lejlighed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Stor 3 værelses lejlighed beliggende på Annaga...,https://www.boligportal.dk/lejligheder/kors%C3...
4,"Ramsvej, 8800 Viborg - 2. sal",6990.0,1000.0,100.0,3.0,2.0,Lejlighed,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Rigtig fin lejlighed med central beliggenhed ...,https://www.boligportal.dk/lejligheder/viborg/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14233,"Reberbansgade, 9000 Aalborg, Aalborg - 3. sal",4875.0,500.0,56.0,2.0,3.0,Lejlighed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Dejlig 2 værelses lejlighed i centralt beligge...,https://www.boligportal.dk/lejligheder/aalborg...
14234,"Parkvej, 4700 Næstved - 5. sal",6500.0,700.0,48.0,2.0,5.0,Lejlighed,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,Lejligheden har en super beliggenhed og udleje...,https://www.boligportal.dk/lejligheder/n%C3%A6...
14235,"Skolegade, 9300 Sæby",4200.0,700.0,83.0,2.0,NaN,Lejlighed,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,Stor rummelig 2-værelses lejlighed med eget kø...,https://www.boligportal.dk/lejligheder/s%C3%A6...
14236,"Danmarksgade, 9000 Aalborg, Aalborg - 2. sal",5675.0,600.0,97.0,3.0,2.0,Lejlighed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Særdeles velbeliggende 3-vær. midt i Aalborg.\...,https://www.boligportal.dk/lejligheder/aalborg...


In [ ]:
# Count the occurrences of "Lejlighed" in the "Property type" column
lejlighed_count = (bp_df['Property type'] == 'Lejlighed').sum()

# Print the count
print("Number of 'Lejligheder':", lejlighed_count)

In [ ]:
import statsmodels.api as sm

# Drop rows with NaN and '-' values in 'Monthly rent', 'Square meters', 'Floor', and 'Rooms' columns
columns_to_clean = ['Monthly rent', 'Square meters', 'Floor', 'Rooms', 'Aconto']
for column in columns_to_clean:
    bp_df = bp_df[bp_df[column] != '-']
    bp_df.dropna(subset=[column], inplace=True)

# Convert columns to numeric data types, replacing commas with periods
for column in columns_to_clean:
    bp_df[column] = pd.to_numeric(bp_df[column].str.replace(',', '.'))

# Define the predictor variables (features)
X = bp_df[['Square meters', 'Floor', 'Rooms', 'Aconto']]

# Add a constant term to the predictor variables
X = sm.add_constant(X)

# Define the target variable
y = bp_df['Monthly rent']

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the regression summary
print(model.summary())
